# Collabrative Filtering (Low Rank Matrix Factorization)

The goal is to fill in entries that are undefined. I use the following notations:
- $x^{(i)}$ as latent variables for item $i$, where $x^{(i)}\in \mathbb{R}^n$, and $i\in\{1,\cdots,n_i\}$
- $\theta^{(j)}$ as corresponding parameters for user $j$, where $\theta^{(j)}\in \mathbb{R}^n$, and $j\in\{1,\cdots,n_u\}$
- $y^{(i,j)}$ as rating for item $i$ and user $j$

Cost function with $L_2$ regularization is
$$J(x^{(1)},\cdots,x^{(n_i)},\theta^{(1)},\cdots,\theta^{(n_u)})=\frac{1}{2}\sum_{(i,j):y^{(i,j)}\in \mathbb{R}} \left(\theta^{(j)\top} x^{(i)}-y^{(i,j)}\right)^2 + \frac{\lambda}{2} \left(\sum_{i=1}^{n_i}\lVert x^{(i)} \rVert^2 + \sum_{j=1}^{n_u}\lVert \theta^{(j)} \rVert^2 \right) $$

I stack $x^{(i)}$ by rows (i.e. $X\in \mathbb{R}^{n_i \times n}$), $\theta^{(j)}$ by rows (i.e. $\Theta\in \mathbb{R}^{n_u \times n}$) for vectorized computation.

In [1]:
import numpy as np
import pandas as pd

In [2]:
def compute_cost(X, Theta, Y, has_rating, reg):
    error = (X @ Theta.T - Y) * has_rating
    squared_diff = np.sum(error @ error.T) / 2
    penalty = reg / 2 * (np.linalg.norm(X) ** 2 + np.linalg.norm(Theta) ** 2)
    return squared_diff + penalty

## Gradient Descent

From the cost function,
$$\frac{\partial J}{\partial x^{(i)}} = \sum_{j:y^{(i,j)}\in \mathbb{R}} \left(\theta^{(j)\top} x^{(i)}-y^{(i,j)}\right)\theta^{(j)} + \lambda x^{(i)}$$
$$\frac{\partial J}{\partial \theta^{(j)}} = \sum_{i:y^{(i,j)}\in \mathbb{R}} \left(\theta^{(j)\top} x^{(i)}-y^{(i,j)}\right)x^{(i)} + \lambda \theta^{(j)}$$

In [3]:
def update(X, Theta, Y, has_rating, reg, alpha):
    ni, nu = Y.shape[0], Y.shape[1]
    dX = ((X @ Theta.T - Y) * has_rating) @ Theta + reg * X
    dTheta =  ((X @ Theta.T - Y) * has_rating).T @ X + reg * Theta   
    X -= alpha * dX
    Theta -= alpha * dTheta
    return X, Theta 

In [4]:
def train(Y, has_rating, reg, alpha, n_factor, n_iter):
    ni, nu = Y.shape[0], Y.shape[1]
    # randomly initialize parameters for symmetry breaking
    X = np.random.randn(ni, n_factor)
    Theta = np.random.randn(nu, n_factor)
    for i in range(n_iter):
        if i == 0 or i % 100 == 0:
            print("Iteration {}, loss = {}".format(i, compute_cost(X, Theta, Y, has_rating, reg)))
        # gradient descent
        X, Theta = update(X, Theta, Y, has_rating, reg, alpha)
    return X, Theta

## Application: Movie Ratings

Load movie ratings. Rating ranges from 1 to 10. 0 means movie is not rated.

In [5]:
dat = pd.read_csv("movie.csv", header=None)
Y = dat.values
has_rating = (Y > 0)
print(Y.shape)

(1682, 943)


Fit available ratings without regularization

In [6]:
np.random.seed(915)
X_fitted, Theta_fitted = train(Y, has_rating, 0, 0.0009, 10, 2501)

Iteration 0, loss = 121266759.78751318
Iteration 100, loss = 29853.05265509899
Iteration 200, loss = 9507.080074612975
Iteration 300, loss = 4896.117034932882
Iteration 400, loss = 3121.577813294124
Iteration 500, loss = 2259.130559020625
Iteration 600, loss = 1781.5022411073467
Iteration 700, loss = 1494.7530935201921
Iteration 800, loss = 1312.2943404808077
Iteration 900, loss = 1190.5847241946597
Iteration 1000, loss = 1105.724812729537
Iteration 1100, loss = 1043.7899094980621
Iteration 1200, loss = 996.4518840409312
Iteration 1300, loss = 958.661426848383
Iteration 1400, loss = 927.301442458483
Iteration 1500, loss = 900.4014693029166
Iteration 1600, loss = 876.681937668676
Iteration 1700, loss = 855.2921660614953
Iteration 1800, loss = 835.6617893110274
Iteration 1900, loss = 817.4141963643846
Iteration 2000, loss = 800.3104936752751
Iteration 2100, loss = 784.2080712700907
Iteration 2200, loss = 769.0277950783516
Iteration 2300, loss = 754.7284634233038
Iteration 2400, loss = 74

Available Ratings

In [7]:
Y

array([[5, 4, 0, ..., 5, 0, 0],
       [3, 0, 0, ..., 0, 0, 5],
       [4, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

Predicted Ratings

In [8]:
X_fitted @ Theta_fitted.T

array([[ 4.06435982,  4.40733571,  2.86958667, ...,  4.18291577,
         4.44027669,  3.73750388],
       [ 3.26669642,  3.53034109,  3.49442406, ...,  2.58727408,
         3.82576191,  3.37913655],
       [ 2.78499772,  1.51880794,  1.98791109, ...,  2.48822526,
         2.93909163,  3.73325698],
       ...,
       [ 1.35588404,  1.65099614, -0.22001495, ...,  3.3798292 ,
         1.09555323,  0.7815146 ],
       [ 2.63745887,  3.90036318,  4.05913082, ...,  4.90007577,
         5.70655651,  0.99602897],
       [ 3.88122052,  4.64577864,  6.34450992, ...,  1.43261582,
         2.40620525,  2.62535751]])